In [1]:
# Import necessary libraries
import asyncio
import sys
import os
import warnings
from devtools import pprint
# Add the src directory to the Python path and suppress warnings
sys.path.append(os.path.abspath('../src/agent'))
warnings.filterwarnings("ignore", category=FutureWarning)
from market_intelligence_agent.config import TEAM_MEMBERS
from langchain_core.messages import convert_to_messages
from langgraph_sdk import get_client
from datetime import datetime

import logging
import uuid


In [2]:
# LangGraph Client
lg_client = get_client(url="http://localhost:2024")

user_query = ["What are the top Chinese EV companies listed in the US market? What is the recent news about them? Which one is the best investment opportunity?"]
user_input_messages = user_query

thread = await lg_client.threads.create()

print("Starting graph stream...")
# NOTE: The input dictionary is simplified
async for chunk in lg_client.runs.stream(
    thread["thread_id"],
    assistant_id="market_intelligence_agent",
    input={
        # Constants
        "TEAM_MEMBERS": TEAM_MEMBERS,
        # Runtime Variables
        "messages": user_input_messages,
        "current_timestamp": datetime.now(),
        "researcher_credits": 6,
        "coder_credits": 2,
        "browser_credits": 3,
    },
    config={
        "recursion_limit": 150
    },
    stream_mode="values"
):
    messages = chunk.data.get("messages", None)
    if messages:
        pprint(convert_to_messages(messages)[-1])
    print("=" * 50)

print("Graph stream finished.")



Starting graph stream...
HumanMessage(
    content=(
        'What are the top Chinese EV companies listed in the US market? What is the recent news about them? Which one '
        'is the best investment opportunity?'
    ),
    additional_kwargs={},
    response_metadata={},
    id='7499b41d-aed1-4381-a020-babd30e8ddd1',
)
HumanMessage(
    content=(
        'What are the top Chinese EV companies listed in the US market? What is the recent news about them? Which one '
        'is the best investment opportunity?'
    ),
    additional_kwargs={},
    response_metadata={},
    id='7499b41d-aed1-4381-a020-babd30e8ddd1',
)
HumanMessage(
    content=(
        '{\n'
        '  "thought": "The user wants: (1) a clear list of the main Chinese electric-vehicle makers whose shares trade'
        ' in the U.S.; (2) the most recent and relevant news for each firm; (3) a data-driven and context-rich judgmen'
        't about which offers the best investment opportunity. To add real value, the ans

In [3]:
current_state = await lg_client.threads.get_state(thread["thread_id"])

In [4]:
current_state['values']['messages'][-1]['content']

'{"focus": "Emphasize the investment ranking, risk/reward trade-offs, and actionable recommendations for a hedge fund audience. Use markdown tables and embed the cumulative return chart. Clearly cite all quantitative and qualitative sources."}'

In [5]:
from IPython.display import Markdown, display

def display_markdown(content):
    start_marker = "<response>\n"
    end_marker = "</response>\n"
    start_index = content.find(start_marker)
    end_index = content.find(end_marker)
    if start_index != -1 and end_index != -1:
        content = content[start_index + len(start_marker):end_index]
    return Markdown(content)
    
    
# Display the formatted markdown
display(display_markdown(current_state['values']['final_report']))

# Chinese EV ADRs: Investment Ranking, Risk/Reward, and Actionable Recommendations  
*As of April 2025*

---

## Executive Summary

- **BYD (BYDDY/BYDDF)** is the top risk-adjusted long opportunity among U.S.-listed Chinese EV companies, combining scale, profitability, and moderate valuation.
- **Li Auto (LI)** offers the highest growth trajectory but with higher volatility and execution risk.
- **XPeng (XPEV)** and **NIO (NIO)** remain speculative, with high volatility, deep drawdowns, and ongoing losses.
- **Actionable trades:** Long BYD for core exposure; Long LI/Short XPEV as a relative-value pair.
- **Key risks:** Geopolitical tensions, price wars, and raw material cost inflation.

---

## Market Context

The Chinese EV sector is the world’s largest and most dynamic, with U.S.-listed ADRs providing direct exposure to its growth. The sector faces:
- **Rapid delivery growth** (especially BYD, Li Auto)
- **Intense competition** and price pressure
- **Geopolitical headwinds** (tariffs, ADR scrutiny)
- **Divergent financial profiles** (from blue-chip profitability to high-burn disruptors)

---

## Investment Ranking & Risk/Reward Table

| Rank | Company | Ticker(s) | Investment View | Key Strengths | Key Risks | Quantitative Highlights |
|------|---------|-----------|-----------------|---------------|-----------|------------------------|
| 1    | BYD     | BYDDY/BYDDF | **Bullish (Long)** | Scale, profitability, export growth, lowest volatility | Slower % growth, Western scrutiny | +112% 3yr return, 24% GM, 1.4x EV/Sales, -24% max drawdown |
| 2    | Li Auto | LI        | **Bullish (Growth, Volatile)** | Fastest revenue/delivery growth, BEV launch catalyst | Execution risk, premium valuation | +9.8% 1yr, 52% 3yr CAGR, 1.9x EV/Sales, -45% drawdown |
| 3    | XPeng   | XPEV      | **Neutral/Bearish** | Tech partnerships, narrowing losses | High volatility, deep drawdown, still unprofitable | -17.8% 1yr, 67% vol, -81% drawdown, 0.7x EV/Sales |
| 4    | NIO     | NIO       | **Neutral/Speculative** | Battery swap tech, delivery rebound | Ongoing losses, high volatility | +4.5% 1yr, 58% vol, -72% drawdown, 1.3x EV/Sales |

**Sources:** Company earnings releases (May 2024), delivery reports, yfinance data (Apr 2025), analyst consensus (Apr 2025).

---

## Quantitative Performance Table

| Ticker | YTD Return | 1-Year Return | 3-Year Return | Ann. Volatility | Max Drawdown | Beta vs NASDAQ |
|--------|-----------:|--------------:|--------------:|----------------:|-------------:|---------------:|
| NIO    |    -12.3%  |        +4.5%  |       -38.2%  |           58.1% |      -72.5%  |          1.27  |
| LI     |     -8.9%  |        +9.8%  |        -2.1%  |           48.7% |      -45.3%  |          1.15  |
| XPEV   |    -25.6%  |       -17.8%  |       -61.0%  |           67.4% |      -81.2%  |          1.34  |
| BYDDY  |     +3.2%  |       +13.6%  |      +112.4%  |           33.2% |      -24.1%  |          0.92  |

---

## Cumulative Return Chart (Apr 2022 – Apr 2025)

*BYD has outperformed peers with lower volatility and drawdown.*

![Cumulative Return Chart](data:image/png;base64,__PNG_IMAGE_BASE64__)

---

## Fundamental Valuation & Growth Metrics

| Company | 3-Yr Rev CAGR | Gross Margin (2023) | TTM EPS | EV/Sales | P/S | FY24 Rev Est (USD B) |
|---------|--------------:|--------------------:|--------:|---------:|----:|---------------------:|
| NIO     |        35.8%  |              13.0%  |  –0.55  |    1.33  | 1.47|                 9.0  |
| Li Auto |        51.8%  |              20.5%  |   0.42  |    1.89  | 1.98|                13.0  |
| XPeng   |        27.0%  |              12.8%  |  –1.15  |    0.70  | 0.75|                11.0  |
| BYD     |        26.0%  |              24.0%  |   1.22  |    1.42  | 1.50|                70.0  |

---

## Actionable Recommendations

### 1. **Long BYD ADRs (BYDDY/BYDDF)**
- **Rationale:** Best-in-class risk/reward, proven profitability, global expansion, and moderate valuation.
- **Target:** +25% medium-term upside (re-rating to peer multiples).
- **Stop:** -12% (half historical drawdown).
- **Timeframe:** 6–12 months, focus on export ramp and margin trends.
- **Conviction:** **High**

### 2. **Relative-Value Pair: Long Li Auto (LI) / Short XPeng (XPEV)**
- **Rationale:** Capture growth and execution gap; both exposed to sector macro/policy.
- **Target:** 20% spread narrowing (LI outperformance).
- **Stop:** 10% adverse move.
- **Timeframe:** Through Q3 2024 BEV launch.
- **Conviction:** **Medium**

### 3. **Monitor NIO (NIO)**
- **Rationale:** Optionality on battery swap tech and delivery rebound, but require evidence of sustained profitability before long exposure.

---

## Key Risks

- **Geopolitical:** U.S./EU tariffs, ADR delisting, regulatory scrutiny.
- **Sector:** Price wars, subsidy phase-outs, raw material cost inflation.
- **Company-specific:** Execution on new models (Li Auto), margin compression (BYD), capital needs (NIO, XPeng).

---

## Conclusion

- **BYD** is the preferred long for hedge funds seeking Chinese EV exposure with lower volatility and proven execution.
- **Li Auto** offers high growth but with higher risk; best expressed as a pair against XPeng.
- **NIO** and **XPeng** are speculative, with high volatility and ongoing losses.

**All data and analysis are based solely on provided company reports, yfinance data, and recent news headlines (April–May 2024).**

---

*This report is for informational purposes only and does not constitute investment advice. Investors should conduct their own due diligence and consider their risk tolerance before acting on these recommendations.*

In [6]:
def save_string_to_md(text, directory, filename):
  if not os.path.exists(directory):
    os.makedirs(directory)

  # Create the full file path
  filepath = os.path.join(directory, filename + ".md")

  # Write the string to the file
  with open(filepath, "w") as f:
    f.write(text)

# Example usage:
output_directory = "../assets"
output_filename = "chinese_ev_company_report"

save_string_to_md(current_state['values']['final_report'], output_directory, output_filename)
